# SSIS Analyzer
- Analyze connection manager in SSIS project
- Analyze source/target in Data Flow components
- Analyze Stored Procedure and its source/traget in SQL Task 

SSIS Analyzer is aim to generate a data lineage when using SSIS as ETL tool

# Features
- SSIS project directory is full managed by SQL Server File Table
- Using SQL Server XML XQuery to analyze SSIS package
- Using system views to get the Stored Procddure reference
- SSIS Analyzer can be deployed on SQL Server without any extra reference

# Create File Table

In [0]:
--SSIS Connection Manager
CREATE TABLE [ft].[ETL_SSIS_CONNECTIONS] AS FILETABLE ON [FG_DATA] FILESTREAM_ON [FSG_1]
WITH
(
FILETABLE_DIRECTORY = N'ETL_SSIS_CONNECTIONS', FILETABLE_COLLATE_FILENAME = SQL_Latin1_General_CP1_CI_AS
)
GO

--SSIS Package
CREATE TABLE [ft].[ETL_SSIS_PACKAGES] AS FILETABLE ON [FG_DATA] FILESTREAM_ON [FSG_1]
WITH
(
FILETABLE_DIRECTORY = N'ETL_SSIS_PACKAGES', FILETABLE_COLLATE_FILENAME = SQL_Latin1_General_CP1_CI_AS
)
GO

# Create other tables

In [1]:
--存放XML转义字符清单
CREATE TABLE [dbo].[XML_ESCAPE](
	[XML_ESCAPE_ID] [int] IDENTITY(1,1) NOT NULL,
	[TEXT_CHAR] [nvarchar](5) NOT NULL,
	[XML_CHAR] [nvarchar](10) NOT NULL,
PRIMARY KEY CLUSTERED 
(
	[XML_ESCAPE_ID] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
) ON [FG_DATA]
GO

--手动补充维护SP与目标Table的关系
CREATE TABLE [dbo].[ETL_SP_MANUAL_TARGET_TABLE_MAPPING](
	[SP_DATABASE_NAME] [sysname] NOT NULL,
	[SP_SCHEMA_NAME] [sysname] NOT NULL,
	[SP_NAME] [sysname] NOT NULL,
	[TARGET_DATABASE_NAME] [sysname] NOT NULL,
	[TARGET_SCHEMA_NAME] [sysname] NOT NULL,
	[TARGET_TABLE_NAME] [sysname] NOT NULL,
	[COMMENT] [nvarchar](500) NULL
) ON [FG_DATA]
GO

--SP与目标Table的关系(包括系统判断与手动补充维护)
CREATE TABLE [dbo].[ETL_SP_TARGET_TABLE_MAPPING](
	[SP_TARGET_TABLE_MAPPING_ID] [int] IDENTITY(1,1) NOT NULL,
	[SP_DATABASE_NAME] [sysname] NOT NULL,
	[SP_SCHEMA_NAME] [sysname] NOT NULL,
	[SP_NAME] [sysname] NOT NULL,
	[TARGET_DATABASE_NAME] [sysname] NOT NULL,
	[TARGET_SCHEMA_NAME] [sysname] NOT NULL,
	[TARGET_TABLE_NAME] [sysname] NOT NULL,
	[IS_SYSTEM] [bit] NOT NULL,
	[COMMENT] [nvarchar](500) NULL,
 CONSTRAINT [PK_ETL_SP_TARGET_TABLE_MAPPING] PRIMARY KEY CLUSTERED 
(
	[SP_TARGET_TABLE_MAPPING_ID] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
) ON [FG_DATA]
GO


--SP与引用对象的关系
CREATE TABLE [dbo].[ETL_SP_REFERENCED_TABLE_MAPPING](
	[SP_REFERENCED_TABLE_MAPPING_ID] [int] IDENTITY(1,1) NOT NULL,
	[SP_DATABASE_NAME] [sysname] NOT NULL,
	[SP_SCHEMA_NAME] [sysname] NOT NULL,
	[SP_NAME] [sysname] NOT NULL,
	[REFERENCED_DATABASE_NAME] [sysname] NOT NULL,
	[REFERENCED_SCHEMA_NAME] [sysname] NOT NULL,
	[REFERENCED_TABLE_NAME] [sysname] NOT NULL
) ON [FG_DATA]

--存放SSIS Connection对象
CREATE TABLE [dbo].[ETL_SSIS_CONNECTIONS](
	[SSIS_CONNECTION_ID] [int] IDENTITY(1,1) NOT NULL,
	[SSIS_CONNECTION_GUID] [nchar](38) NOT NULL,
	[SSIS_CONNECTION_NAME] [nvarchar](200) NOT NULL,
	[SSIS_CONNECTION_TYPE] [nvarchar](100) NOT NULL,
	[SSIS_CONNECTION_STRING_VARIABLE] [nvarchar](100) NULL,
	[AUTHENTICATION] [nvarchar](20) NOT NULL,
	[SERVER_NAME] [nvarchar](200) NULL,
	[DATABASE_NAME] [nvarchar](128) NULL,
	[LOGIN_NAME] [nvarchar](128) NULL,
 CONSTRAINT [PK_ETL_SSIS_SQL_CONNECTIONS] PRIMARY KEY CLUSTERED 
(
	[SSIS_CONNECTION_ID] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
) ON [FG_DATA]
GO

--存放SSIS Package解析对象
CREATE TABLE [dbo].[ETL_SSIS_PACKAGE_ANALYSIS](
	[PACKAGE_ANALYSIS_ID] [int] IDENTITY(1,1) NOT NULL,
	[PACKAGE_NAME] [nvarchar](200) NOT NULL,
	[CORE_COMPONENT_ID] [int] NOT NULL,
	[CORE_COMPONENT_TYPE] [nvarchar](50) NOT NULL,
	[CORE_COMPONENT_NAME] [nvarchar](200) NOT NULL,
	[SOURCE_ID] [int] NULL,
	[SOURCE_CONNECTION] [nvarchar](50) NULL,
	[SOURCE_TYPE] [nvarchar](50) NULL,
	[SOURCE_TEXT] [nvarchar](max) NULL,
	[TARGET_ID] [int] NULL,
	[TARGET_CONNECTION] [nvarchar](50) NULL,
	[TARGET_TYPE] [nvarchar](50) NULL,
	[TARGET_TEXT] [nvarchar](200) NULL,
PRIMARY KEY CLUSTERED 
(
	[PACKAGE_ANALYSIS_ID] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
) ON [FG_DATA] TEXTIMAGE_ON [FG_DATA]
GO

--存放SSIS Package.Data Flow.Source SELECT SQL语句解析对象
CREATE TABLE [dbo].[ETL_SSIS_PACKAGE_SQL_SELECT_ANALYSIS](
	[PACKAGE_SQL_SELECT_ANALYSIS_ID] [int] IDENTITY(1,1) NOT NULL,
	[PACKAGE_ANALYSIS_ID] [int] NOT NULL,
	[REFERENCED_DATABASE_NAME] [nvarchar](128) NOT NULL,
	[REFERENCED_SCHEMA_NAME] [nvarchar](128) NOT NULL,
	[REFERENCED_TABLE_NAME] [nvarchar](128) NOT NULL,
PRIMARY KEY CLUSTERED 
(
	[PACKAGE_SQL_SELECT_ANALYSIS_ID] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
) ON [FG_DATA]
GO

--存放SSIS Package.SQL Task Stored Procedure解析对象
CREATE TABLE [dbo].[ETL_SSIS_PACKAGE_SQL_TASK_ANALYSIS](
	[PACKAGE_SQL_TASK_ANALYSIS_ID] [int] IDENTITY(1,1) NOT NULL,
	[PACKAGE_ANALYSIS_ID] [int] NOT NULL,
	[REFERENCED_DATABASE_NAME] [nvarchar](128) NOT NULL,
	[REFERENCED_SCHEMA_NAME] [nvarchar](128) NOT NULL,
	[REFERENCED_SP_NAME] [nvarchar](128) NOT NULL,
PRIMARY KEY CLUSTERED 
(
	[PACKAGE_SQL_TASK_ANALYSIS_ID] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
) ON [FG_DATA]
GO

--存放SSIS Package最终解析结果
CREATE TABLE [dbo].[ETL_SSIS_PACKAGE_TABLE_LINEAGE](
	[PACKAGE_TABLE_LINEAGE_ID] [int] IDENTITY(1,1) NOT NULL,
	[PACKAGE_NAME] [nvarchar](200) NOT NULL,
	[CORE_COMPONENT_ID] [int] NOT NULL,
	[CORE_COMPONENT_TYPE] [nvarchar](20) NOT NULL,
	[CORE_COMPONENT_NAME] [nvarchar](200) NOT NULL,
	[REFERENCED_DATABASE_NAME] [nvarchar](128) NOT NULL,
	[REFERENCED_SCHEMA_NAME] [nvarchar](128) NULL,
	[REFERENCED_TABLE_NAME] [nvarchar](128) NULL,
	[REFERENCING_DATABASE_NAME] [sysname] NOT NULL,
	[REFERENCING_SCHEMA_NAME] [sysname] NOT NULL,
	[REFERENCING_TABLE_NAME] [sysname] NOT NULL,
	[IS_MAIN_REFERENCE] [bit] NOT NULL,
 CONSTRAINT [PK_ETL_SSIS_PACKAGE_TABLE_LINEAGE] PRIMARY KEY CLUSTERED 
(
	[PACKAGE_TABLE_LINEAGE_ID] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
) ON [FG_DATA]
GO


# Create Unique Index

In [0]:
CREATE NONCLUSTERED INDEX [UIX_ETL_SP_REFERENCED_TABLE_MAPPING] ON [dbo].[ETL_SP_REFERENCED_TABLE_MAPPING]
(
	[SP_DATABASE_NAME] ASC,
	[SP_SCHEMA_NAME] ASC,
	[SP_NAME] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
GO
SET ANSI_PADDING ON
GO

CREATE NONCLUSTERED INDEX [UIX_ETL_SP_TARGET_TABLE_MAPPING] ON [dbo].[ETL_SP_TARGET_TABLE_MAPPING]
(
	[SP_DATABASE_NAME] ASC,
	[SP_SCHEMA_NAME] ASC,
	[SP_NAME] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
GO
SET ANSI_PADDING ON
GO

CREATE UNIQUE NONCLUSTERED INDEX [UIX_ETL_SSIS_CONNECTION_GUID] ON [dbo].[ETL_SSIS_CONNECTIONS]
(
	[SSIS_CONNECTION_GUID] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [FG_DATA]
GO

In [2]:
--XML转义字符清单
INSERT INTO dbo.XML_ESCAPE
VALUES(N'<', N'&lt;'),(N'>', N'&gt;'),(N'&', N'&amp;'),(N'''', N'&apos;'),(N'"', N'&quot;');

# Create Function UDF_XML_ESCAPE

In [3]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/06
-- Description:    
-- =============================================
CREATE FUNCTION [dbo].[UDF_XML_ESCAPE] 
(
	@TEXT NVARCHAR(MAX)
)
RETURNS NVARCHAR(MAX)
AS
BEGIN
 
	DECLARE @CONVERTED_TEXT NVARCHAR(MAX), @XML_ESCAPE_NUMBER INT, @I INT = 1;
 
	SET @CONVERTED_TEXT = @TEXT;
	SET @XML_ESCAPE_NUMBER = (SELECT COUNT(*) FROM dbo.XML_ESCAPE);
	WHILE (@I <= @XML_ESCAPE_NUMBER)
	BEGIN
 
		SELECT @CONVERTED_TEXT = REPLACE(@CONVERTED_TEXT,XML_CHAR,TEXT_CHAR)   
		FROM dbo.XML_ESCAPE WHERE XML_ESCAPE_ID = @I;
 
		SET @I += 1;
 
	END
 
	RETURN @CONVERTED_TEXT;
 
END

# Create Function UDF_GET_SSIS_DATA_FLOW_CONNECTION

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/10/31
-- Description:    
-- =============================================
CREATE FUNCTION [dbo].[UDF_GET_SSIS_DATA_FLOW_CONNECTION] 
(
	@COMPONENT XML,
	@CLASS NVARCHAR(50)
)
RETURNS NVARCHAR(200)
AS
BEGIN
 
	DECLARE @CONNECTION  NVARCHAR(200),@QUERY_TEXT NVARCHAR(500);
 
	--检索[根节点]component.connections.connection.[属性]name = "OleDbConnection"/"FlatFileConnection".[属性]connectionManagerRefId
 
	IF @CLASS = N'OLEDB'
	BEGIN
		SET @CONNECTION = CONVERT(NVARCHAR(200),@COMPONENT.query('data(/component/connections/connection[@name="OleDbConnection"]/@connectionManagerRefId)'));
	END
	ELSE IF @CLASS = N'FlatFile'
	BEGIN
		SET @CONNECTION = CONVERT(NVARCHAR(200),@COMPONENT.query('data(/component/connections/connection[@name="FlatFileConnection"]/@connectionManagerRefId)'));
	END
 
	RETURN @CONNECTION;
 
END 
 
GO

# Create Fuction UDF_GET_SSIS_DATA_FLOW_ACCESS_MODE

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/10/31
-- Description:    
-- =============================================
CREATE FUNCTION [dbo].[UDF_GET_SSIS_DATA_FLOW_ACCESS_MODE] 
(
	@COMPONENT XML
)
RETURNS INT
AS
BEGIN
 
	DECLARE @ACCESS_MODE  INT;
 
	--检索[根节点]component中子[节点]properties.子[节点]property [属性]name = "AccessMode"
	SET @ACCESS_MODE = CONVERT(INT,CONVERT(NVARCHAR(10),@COMPONENT.query('data(/component/properties/property[@name="AccessMode"])')));
	RETURN @ACCESS_MODE
 
END 
 
GO

# Create Stored Procedure USP_ANALYSE_SSIS_CONNECTION

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/14
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_ANALYSE_SSIS_CONNECTION]
	@DOC_XML XML
AS
BEGIN
 
	SET NOCOUNT ON;
 
	--变量定义
	--===================================================================================================
	--[dbo].[ETL_SSIS_SSIS_CONNECTIONS]
	DECLARE
		@SSIS_CONNECTION_GUID NCHAR(38),
		@SSIS_CONNECTION_NAME NVARCHAR(200),
		@SSIS_CONNECTION_TYPE NVARCHAR(100),
		@SSIS_CONNECTION_STRING_VARIABLE NVARCHAR(100),
 
		@SSIS_CONNECTION_STRING NVARCHAR(500),
		@AUTHENTICATION NVARCHAR(20),
		@SERVER_NAME NVARCHAR(200),
		@DATABASE_NAME NVARCHAR(128),
		@LOGIN_NAME  NVARCHAR(128);
	 
	--===================================================================================================
	--解析XML开始
	--===================================================================================================
	--Connection name
	WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
	SELECT @SSIS_CONNECTION_NAME = CONVERT(NVARCHAR(200),@DOC_XML.query('data(DTS:ConnectionManager/@DTS:ObjectName)')) ;
 
	--Connection GUID with{}
	WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
	SELECT @SSIS_CONNECTION_GUID = CONVERT(NCHAR(38),@DOC_XML.query('data(DTS:ConnectionManager/@DTS:DTSID)')) ;
 
	--Connection type: OLEDB
	WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
	SELECT @SSIS_CONNECTION_TYPE = CONVERT(NVARCHAR(100),@DOC_XML.query('data(DTS:ConnectionManager/@DTS:CreationName)')) ;
 
	--Connection string variable
	WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
	SELECT @SSIS_CONNECTION_STRING_VARIABLE = CONVERT(NVARCHAR(100),@DOC_XML.query('data(DTS:ConnectionManager/DTS:PropertyExpression[@DTS:Name="ConnectionString"])')) ;
 
	--Connection string
	WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
	SELECT @SSIS_CONNECTION_STRING = CONVERT(NVARCHAR(500),@DOC_XML.query('data(DTS:ConnectionManager/DTS:ObjectData/DTS:ConnectionManager/@DTS:ConnectionString)')) ;
 
 
	DECLARE @CONN TABLE (OBJ NVARCHAR(100) NOT NULL);
	INSERT INTO @CONN
	VALUES(N'Data Source'),(N'User ID'),(N'Initial Catalog');
 
	--按分号拆分connection string
	SELECT CO.OBJ,
	SUBSTRING(C.value,CHARINDEX(N'=',C.value)+1,LEN(C.value)-CHARINDEX(N'=',C.value)) AS OBJ_VALUE
	INTO #CO
	FROM 
	STRING_SPLIT(@SSIS_CONNECTION_STRING, ';') AS C
	INNER JOIN
	@CONN AS CO
	ON C.value LIKE N'%' + CO.OBJ + N'%';
 
	SET @SERVER_NAME = (SELECT OBJ_VALUE FROM #CO WHERE OBJ = N'Data Source');
	SET @LOGIN_NAME = (SELECT OBJ_VALUE FROM #CO WHERE OBJ = N'User ID');
	SET @DATABASE_NAME = (SELECT OBJ_VALUE FROM #CO WHERE OBJ = N'Initial Catalog');
	SET @AUTHENTICATION = IIF(@LOGIN_NAME IS NULL,N'Windows',N'SQL');
 
	--插入结果    
	INSERT INTO dbo.ETL_SSIS_CONNECTIONS
	VALUES(@SSIS_CONNECTION_GUID,@SSIS_CONNECTION_NAME,@SSIS_CONNECTION_TYPE,@SSIS_CONNECTION_STRING_VARIABLE,@AUTHENTICATION,@SERVER_NAME,@DATABASE_NAME,@LOGIN_NAME);
 
 
END
GO

# Create Stored Procedure USP_ANALYSE_SSIS_PACKAGE

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/10/31
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_ANALYSE_SSIS_PACKAGE]
	@PACKAGE_NAME NVARCHAR(200),
	@DOC_XML XML
AS
BEGIN
 
	SET NOCOUNT ON;
 
	--变量定义
	--===================================================================================================
	--XML变量
	DECLARE @DATA_FLOW_XML XML, @SOURCE_XML XML, @TARGET_XML XML, @SQL_TASK_XML XML;
	--Data Flow元素变量
	DECLARE
		@DATA_FLOW_NAME NVARCHAR(100),
		@SOURCE_CLASS NVARCHAR(50),
		@SOURCE_ACCESS_MODE NVARCHAR(50),
		@SOURCE_CONNECTION NVARCHAR(200),
		@SOURCE_DB_NAME NVARCHAR(128),
 
		--同一Source中以下3者不共存(仅1个不为空)
		@SOURCE_FILE_NAME NVARCHAR(200),
		@SOURCE_TABLE_NAME NVARCHAR(200),
		@SOURCE_SQL_TEXT NVARCHAR(MAX),
 
		@TARGET_CLASS NVARCHAR(50),
		@TARGET_ACCESS_MODE NVARCHAR(50),
		@TARGET_CONNECTION NVARCHAR(200),
		@TARGET_DB_NAME NVARCHAR(128),
		--同一Target中以下2者不共存(仅1个不为空)
		@TARGET_FILE_NAME NVARCHAR(200),
		@TARGET_TABLE_NAME NVARCHAR(200),
 
		@DATA_FLOW_NUMBER INT, @SQL_TASK_NUMBER INT, @SOURCE_NUMBER INT, @TARGET_NUMBER INT;
	--SQL Task元素变量
	DECLARE 
		@SQL_TASK_CONNECTION_GUID NCHAR(38),
		@SQL_TASK_CONNECTION NVARCHAR(128),
		@SQL_TASK_NAME NVARCHAR(200),
		@SQL_TASK_TEXT NVARCHAR(MAX);
 
	--循环变量
	DECLARE @I INT = 1, @J INT = 1, @K INT = 1, @L INT = 1;
 
	--Source/Target的AccessMode定义
	DECLARE @ACCESS_MODE_MAPPING TABLE 
	(
		OBJECT_TYPE NVARCHAR(50) NOT NULL,
		ACCESS_MODE_ID INT NOT NULL,
		ACCESS_MODE_NAME NVARCHAR(100) NOT NULL
	);
 
	INSERT INTO @ACCESS_MODE_MAPPING VALUES
	(N'Source',0,N'Table or view'),(N'Source',2,N'SqlCommand'),
	(N'Target',0,N'Table or view'),(N'Target',3,N'Table or view - fast load');
 
	--Source/Target的XML集合
	CREATE TABLE #SOURCE_LIST 
	(
		[SOURCE_ID] INT NOT NULL IDENTITY(1,1),
		[SOURCE_CLASS] NVARCHAR(50) NULL,
		[SOURCE_XML] XML
	);
 
	CREATE TABLE #TARGET_LIST 
	(
		[TARGET_ID] INT NOT NULL IDENTITY(1,1),
		[TARGET_CLASS] NVARCHAR(50) NULL,
		[TARGET_XML] XML
	);
 
	--Source/Target元素抽取结果
	DECLARE @SOURCE_RESULT TABLE 
	(
		[SOURCE_ID] [int] NOT NULL,
		[SOURCE_CONNECTION] [nvarchar](50) NULL,
		[SOURCE_TYPE] [nvarchar](50) NULL,
		[SOURCE_TEXT] [nvarchar](MAX) NULL
	);
 
	DECLARE @TARGET_RESULT TABLE 
	(
		[TARGET_ID] [int] NOT NULL,
		[TARGET_CONNECTION] [nvarchar](50) NULL,
		[TARGET_TYPE] [nvarchar](50) NULL,
		[TARGET_TEXT] [nvarchar](MAX) NULL
	);
	--===================================================================================================
	--解析XML开始
	--===================================================================================================
	--模式1 获取Data Flow节点
	--Data Flow可在容器中嵌套,位置不定
	--检索root XML中任意位置Executable节点 属性CreationName = "Microsoft.Pipeline"
	--#DF存储Data Flow集合
	WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
	SELECT
		ROW_NUMBER() OVER(ORDER BY (SELECT 1)) AS DF_ID, 
		T.DATA_FLOW_XML.query('.') AS DATA_FLOW_XML INTO #DF
	FROM @DOC_XML.nodes('//DTS:Executable[@DTS:CreationName="Microsoft.Pipeline"]') AS T(DATA_FLOW_XML);
 
	--模式2 获取顺序执行容器中的执行Sql节点,仅兼容固定模式
	WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
	SELECT
		ROW_NUMBER() OVER(ORDER BY (SELECT 1)) AS ST_ID, 
		T.SQL_TASK_XML.query('.') AS SQL_TASK_XML INTO #ST
	FROM @DOC_XML.nodes('DTS:Executable/DTS:Executables/DTS:Executable/DTS:Executables/DTS:Executable[@DTS:CreationName="Microsoft.ExecuteSQLTask"]') AS T(SQL_TASK_XML);
 
	--模式1,2都不符合时,退出
	IF NOT EXISTS (SELECT * FROM #DF) AND NOT EXISTS (SELECT * FROM #ST)  
	BEGIN
		PRINT(@PACKAGE_NAME + N' : no data flow/SP');
		RETURN;
	END
 
	--处理模式1 Data Flow
	--===================================================================================================
	--循环Data Flow节点
	SET @DATA_FLOW_NUMBER = (SELECT COUNT(*) FROM #DF);
 
	WHILE (@I <= @DATA_FLOW_NUMBER)
	BEGIN
		PRINT(CONCAT(N'Process ', @PACKAGE_NAME, N': Data Flow ', @I));
 
		--循环开始时,初始变量
		SET @DATA_FLOW_XML = NULL;
		SET @DATA_FLOW_NAME = NULL;
		SET @DATA_FLOW_XML = (SELECT DATA_FLOW_XML FROM #DF WHERE DF_ID = @I);
 
		TRUNCATE TABLE #SOURCE_LIST;
		TRUNCATE TABLE #TARGET_LIST;
		--Data Flow Name
		--检索Data Flow中根节点Executable的ObjectName属性
		WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
		SELECT @DATA_FLOW_NAME = CONVERT(NVARCHAR(100), @DATA_FLOW_XML.query('data(/DTS:Executable/@DTS:ObjectName)'));
 
		--Data Flow.Source
		--检索Data Flow中ObjectData.pipeline.components.component节点的属性componentClassID = "Microsoft.OLEDBSource"
		WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)
		INSERT INTO #SOURCE_LIST
		SELECT
			N'OLEDB',
			S.SOUECE_XML.query('.') AS SOUECE_XML
		FROM @DATA_FLOW_XML.nodes('/DTS:Executable/DTS:ObjectData/pipeline/components/component[@componentClassID="Microsoft.OLEDBSource"]') AS S(SOUECE_XML);;
 
		--检索Data Flow中ObjectData.pipeline.components.component节点的属性componentClassID = "Microsoft.FlatFile"
		WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)
		INSERT INTO #SOURCE_LIST
		SELECT
			N'FlatFile',
			S.SOUECE_XML.query('.') AS SOUECE_XML
		FROM @DATA_FLOW_XML.nodes('/DTS:Executable/DTS:ObjectData/pipeline/components/component[@componentClassID="Microsoft.FlatFileSource"]') AS S(SOUECE_XML);;
		--循环处理Source XML
		SET @SOURCE_NUMBER = (SELECT COUNT(*) FROM #SOURCE_LIST);
		DELETE FROM @SOURCE_RESULT;
 
		SET @J = 1;
 
		WHILE (@J <= @SOURCE_NUMBER)
		BEGIN
 
			--循环开始时,初始变量
			SET @SOURCE_XML = NULL;
			SET @SOURCE_CLASS = NULL;
			SET @SOURCE_ACCESS_MODE = NULL;
			SET @SOURCE_CONNECTION = NULL;
			SET @SOURCE_DB_NAME = NULL;
			SET @SOURCE_TABLE_NAME = NULL;
			SET @SOURCE_SQL_TEXT = NULL;
			SELECT @SOURCE_XML = SOURCE_XML, @SOURCE_CLASS = SOURCE_CLASS 
			FROM #SOURCE_LIST WHERE SOURCE_ID = @J;
 
			--分别处理OLEDB/FlatFile
			IF @SOURCE_CLASS IN (N'OLEDB')
			BEGIN
				--Data Flow.Source.AccessMode
				--OLEDB Source具有该属性
				SELECT @SOURCE_ACCESS_MODE = ACCESS_MODE_NAME
				FROM @ACCESS_MODE_MAPPING
				WHERE OBJECT_TYPE = N'Source' AND ACCESS_MODE_ID = [dbo].[UDF_GET_SSIS_DATA_FLOW_ACCESS_MODE](@SOURCE_XML);
				--Data Flow.Source.Content
				--检索Source中子节点properties.property 属性name = "SqlCommand"/"OpenRowset"
				--Flat File的Content从Connection中获取    
				IF @SOURCE_ACCESS_MODE = N'SqlCommand'
				BEGIN
					SET @SOURCE_SQL_TEXT = CONVERT(NVARCHAR(MAX),@SOURCE_XML.query('data(/component/properties/property[@name="SqlCommand"])'));
					--转义XML字符
					SET @SOURCE_SQL_TEXT = [dbo].[UDF_XML_ESCAPE](@SOURCE_SQL_TEXT);
				END
				ELSE IF @SOURCE_ACCESS_MODE = N'Table or view'
				BEGIN
					SELECT @SOURCE_TABLE_NAME = CONVERT(NVARCHAR(200),@SOURCE_XML.query('data(/component/properties/property[@name="OpenRowset"])'));
				END;
 
				--Data Flow.Source.Connection
				--检索Source中Database或FlatFile的Connection
				SELECT @SOURCE_CONNECTION = [dbo].[UDF_GET_SSIS_DATA_FLOW_CONNECTION](@SOURCE_XML,@SOURCE_CLASS);
				SET @SOURCE_DB_NAME = [dbo].[UDF_GET_INNER_STRING](@SOURCE_CONNECTION,N'[',N']');
 
			END
			ELSE IF @SOURCE_CLASS IN (N'FlatFile')
			BEGIN
 
				--Data Flow.Source.AccessMode
				SET @SOURCE_ACCESS_MODE = N'Flat File';
 
				--Data Flow.Source.Connection
				--检索Source中Database或FlatFile的Connection
				SELECT @SOURCE_CONNECTION = [dbo].[UDF_GET_SSIS_DATA_FLOW_CONNECTION](@SOURCE_XML,@SOURCE_CLASS);
 
				--Data Flow.Source.Content
				SET @SOURCE_FILE_NAME = [dbo].[UDF_GET_INNER_STRING](@SOURCE_CONNECTION,N'[',N']');
 
			END
 
			--插入Source元素抽取结果
			INSERT INTO @SOURCE_RESULT
	VALUES(@J,COALESCE(@SOURCE_DB_NAME,@SOURCE_FILE_NAME),@SOURCE_ACCESS_MODE,COALESCE(@SOURCE_TABLE_NAME,@SOURCE_SQL_TEXT,@SOURCE_FILE_NAME));
 
			SET @J += 1;
 
		--结束Source循环
		END;
 
		--获取Data Flow.Target
		--检索Data Flow中ObjectData.pipeline.components.component节点的属性componentClassID = "Microsoft.OLEDBDestination"
		WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
		INSERT INTO #TARGET_LIST
		SELECT 
			N'OLEDB',
			T.TARGET_XML.query('.')
		FROM @DATA_FLOW_XML.nodes('/DTS:Executable/DTS:ObjectData/pipeline/components/component[@componentClassID="Microsoft.OLEDBDestination"]') AS T(TARGET_XML);
		--检索Data Flow中ObjectData.pipeline.components.component节点的属性componentClassID = "Microsoft.FlatFileDestination"
		WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)  
		INSERT INTO #TARGET_LIST
		SELECT 
			N'FlatFile',
			T.TARGET_XML.query('.')
		FROM @DATA_FLOW_XML.nodes('/DTS:Executable/DTS:ObjectData/pipeline/components/component[@componentClassID="Microsoft.FlatFileDestination"]') AS T(TARGET_XML);
 
		--循环处理Target XML
		SET @TARGET_NUMBER = (SELECT COUNT(*) FROM #TARGET_LIST);
		DELETE FROM @TARGET_RESULT;
 
		SET @K = 1;
 
		WHILE (@K <= @TARGET_NUMBER)
		BEGIN
 
			--循环开始时,初始变量
			SET @TARGET_XML = NULL;
			SET @TARGET_CLASS = NULL;
			SET @TARGET_ACCESS_MODE = NULL;
			SET @TARGET_CONNECTION = NULL;
			SET @TARGET_DB_NAME = NULL;
			SET @TARGET_TABLE_NAME = NULL;
			SELECT @TARGET_XML = TARGET_XML, @TARGET_CLASS = TARGET_CLASS 
			FROM #TARGET_LIST WHERE TARGET_ID = @K;
 
			--分别处理OLEDB/FlatFile
			IF @TARGET_CLASS IN (N'OLEDB')
			BEGIN
 
				--Data Flow.Target.AccessMode
				SELECT @TARGET_ACCESS_MODE = ACCESS_MODE_NAME
				FROM @ACCESS_MODE_MAPPING
				WHERE OBJECT_TYPE = N'Target' AND ACCESS_MODE_ID = [dbo].[UDF_GET_SSIS_DATA_FLOW_ACCESS_MODE](@TARGET_XML);
 
				--Data Flow.Target.Content
				IF @TARGET_ACCESS_MODE IN (N'Table or view',N'Table or view - fast load')
				BEGIN
					SELECT @TARGET_TABLE_NAME = CONVERT(NVARCHAR(200),@TARGET_XML.query('data(/component/properties/property[@name="OpenRowset"])'));
				END
 
				--Data Flow.Target.Connection
				--检索Target中子节点connections.子节点connection 属性name = "OleDbConnection"的属性connectionManagerRefId
				SELECT @TARGET_CONNECTION = [dbo].[UDF_GET_SSIS_DATA_FLOW_CONNECTION](@TARGET_XML,@TARGET_CLASS);
				SET @TARGET_DB_NAME = [dbo].[UDF_GET_INNER_STRING](@TARGET_CONNECTION,N'[',N']');
 
			END
			ELSE IF @TARGET_CLASS IN (N'FlatFile')
			BEGIN
				SET @TARGET_ACCESS_MODE = N'Flat File';
 
				--Data Flow.Target.Connection
				--检索Target中Database或FlatFile的Connection
				SELECT @TARGET_CONNECTION = [dbo].[UDF_GET_SSIS_DATA_FLOW_CONNECTION](@TARGET_XML,@TARGET_CLASS);
 
				--Data Flow.Source.Content
				SET @TARGET_FILE_NAME = [dbo].[UDF_GET_INNER_STRING](@TARGET_CONNECTION,N'[',N']');
 
			END
 
			--插入Target元素抽取结果
			INSERT INTO @TARGET_RESULT
	VALUES(@K,COALESCE(@TARGET_DB_NAME,@TARGET_FILE_NAME),@TARGET_ACCESS_MODE,COALESCE(@TARGET_TABLE_NAME,@TARGET_FILE_NAME));
			SET @K += 1;
 
		--结束Target循环
		END
 
		--Source join Target
		INSERT INTO dbo.ETL_SSIS_PACKAGE_ANALYSIS([PACKAGE_NAME], [CORE_COMPONENT_ID], [CORE_COMPONENT_TYPE], [CORE_COMPONENT_NAME], [SOURCE_ID], [SOURCE_CONNECTION], [SOURCE_TYPE], [SOURCE_TEXT], [TARGET_ID], [TARGET_CONNECTION], [TARGET_TYPE], [TARGET_TEXT])
		SELECT DISTINCT
			@PACKAGE_NAME,@I,N'Data Flow',@DATA_FLOW_NAME,
	S.SOURCE_ID,S.SOURCE_CONNECTION,S.SOURCE_TYPE,S.SOURCE_TEXT,
	T.TARGET_ID,T.TARGET_CONNECTION,T.TARGET_TYPE,T.TARGET_TEXT
		FROM @SOURCE_RESULT AS S CROSS JOIN @TARGET_RESULT AS T;
 
		SET @I += 1;
 
	--结束Data Flow循环
	END
	--===================================================================================================
 
	--处理模式2 SQL Task
	--===================================================================================================
	SET @SQL_TASK_NUMBER = (SELECT COUNT(*) FROM #ST);
	WHILE (@L <= @SQL_TASK_NUMBER)
	BEGIN
 
		PRINT(CONCAT(N'Process ', @PACKAGE_NAME, N': SQL Task ', @L));
 
		--循环开始时,初始变量
		SET @SQL_TASK_XML = NULL;
		SET @SQL_TASK_CONNECTION = NULL;
		SET @SQL_TASK_CONNECTION_GUID = NULL;
		SET @SQL_TASK_NAME = NULL;
		SET @SQL_TASK_TEXT = NULL;
		SET @SQL_TASK_XML = (SELECT SQL_TASK_XML FROM #ST WHERE ST_ID = @L);
 
		--获取Connection
		WITH XMLNAMESPACES ('www.microsoft.com/sqlserver/dts/tasks/sqltask' AS SQLTask,'www.microsoft.com/SqlServer/Dts' AS DTS)
		SELECT @SQL_TASK_CONNECTION_GUID = CONVERT(NCHAR(38),@SQL_TASK_XML.query('data(DTS:Executable/DTS:ObjectData/SQLTask:SqlTaskData/@SQLTask:Connection)'));
 
		----获取SQL Text
		WITH XMLNAMESPACES ('www.microsoft.com/sqlserver/dts/tasks/sqltask' AS SQLTask,'www.microsoft.com/SqlServer/Dts' AS DTS)
		SELECT @SQL_TASK_TEXT = CONVERT(NVARCHAR(MAX),@SQL_TASK_XML.query('data(DTS:Executable/DTS:ObjectData/SQLTask:SqlTaskData/@SQLTask:SqlStatementSource)'));
 
		--获取SQL task name
		WITH XMLNAMESPACES ('www.microsoft.com/SqlServer/Dts' AS DTS)
		SELECT @SQL_TASK_NAME = CONVERT(NVARCHAR(200),@SQL_TASK_XML.query('data(DTS:Executable/@DTS:ObjectName)'));
 
		--获取Connection DB name
		SET @SQL_TASK_CONNECTION =
		(
			SELECT [DATABASE_NAME] FROM dbo.ETL_SSIS_CONNECTIONS
			WHERE SSIS_CONNECTION_GUID = @SQL_TASK_CONNECTION_GUID
		);
		INSERT INTO dbo.ETL_SSIS_PACKAGE_ANALYSIS([PACKAGE_NAME], [CORE_COMPONENT_ID], [CORE_COMPONENT_TYPE], [CORE_COMPONENT_NAME], [SOURCE_ID], [SOURCE_CONNECTION], [SOURCE_TYPE], [SOURCE_TEXT], [TARGET_ID], [TARGET_CONNECTION], [TARGET_TYPE], [TARGET_TEXT])
		VALUES(@PACKAGE_NAME,1,N'SQL Task',@SQL_TASK_NAME,
		1,@SQL_TASK_CONNECTION,N'SQL Task',@SQL_TASK_TEXT,
		0,'','','');
 
		SET @L += 1;
		 
	END
	--===================================================================================================
 
END
 
 
GO


# Create Stored Procedure USP_ANALYSE_SSIS_SQL_SELECT_TEXT

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/06
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_ANALYSE_SSIS_SQL_SELECT_TEXT]
	@PACKAGE_ANALYSIS_ID INT,
	@SELECT_SQL_TEXT NVARCHAR(MAX),
	@SOURCE_DB NVARCHAR(128)
AS
BEGIN
 
	SET NOCOUNT ON;
 
	 
	DECLARE 
		@DROP_SQL_TEXT NVARCHAR(MAX), 
		@INSERT_SQL_TEXT NVARCHAR(MAX),
		@SQL_EXEC_TEXT NVARCHAR(MAX), 
		@PARM_DEFINITION NVARCHAR(1000),
		@PARM_DEFINITION_VIEW NVARCHAR(1000),
 
		@TEMP_VIEW_NAME NCHAR(36),
		@PACKAGE_NAME NVARCHAR(200),
		@DATA_FLOW_NAME NVARCHAR(200);
 
 
	SET @TEMP_VIEW_NAME = NEWID();    
	SET @SQL_EXEC_TEXT = CONCAT(N'EXEC ', @SOURCE_DB,'.sys.sp_executesql @SQL_TEXT');
	SET @PARM_DEFINITION = N'@SQL_TEXT NVARCHAR(MAX)';
	SET @PARM_DEFINITION_VIEW = N'@TEMP_VIEW_NAME NVARCHAR(38)';
 
	--Create temp view
	SET @SELECT_SQL_TEXT = CONCAT(N'CREATE VIEW test.', QUOTENAME(@TEMP_VIEW_NAME) , N' AS', CHAR(10) , CHAR(13) , @SELECT_SQL_TEXT);
	--PRINT @SELECT_SQL_TEXT
	EXEC sys.sp_executesql @SQL_EXEC_TEXT,@PARM_DEFINITION,@SQL_TEXT = @SELECT_SQL_TEXT;
  
	--Insert into source sql table
	SET @INSERT_SQL_TEXT = CONCAT(
	N'
	INSERT INTO [dbo].[ETL_SSIS_PACKAGE_SQL_SELECT_ANALYSIS]
	SELECT DISTINCT ',@PACKAGE_ANALYSIS_ID,
	N'  
	,ISNULL(D.referenced_database_name,N''{SOURCE_DB}'') AS referenced_database_name,
	ISNULL(referenced_schema_name,N''dbo'') AS referenced_schema_name,
		referenced_entity_name 
	FROM 
	{SOURCE_DB}.sys.sql_expression_dependencies AS D
	INNER JOIN
	{SOURCE_DB}.sys.views AS V
	ON D.referencing_id = V.object_id
	WHERE V.name =''',@TEMP_VIEW_NAME,'''') ;
 
	SET @INSERT_SQL_TEXT = REPLACE(@INSERT_SQL_TEXT,N'{SOURCE_DB}',@SOURCE_DB);
 
	--PRINT @INSERT_SQL_TEXT;
 
	EXEC sys.sp_executesql @INSERT_SQL_TEXT,@PARM_DEFINITION_VIEW,@TEMP_VIEW_NAME = @TEMP_VIEW_NAME;
 
	--Drop temp view
	SET @DROP_SQL_TEXT = N'DROP VIEW test.' + QUOTENAME(@TEMP_VIEW_NAME);
	--PRINT(@DROP_SQL_TEXT);
	EXEC sys.sp_executesql @SQL_EXEC_TEXT,@PARM_DEFINITION,@SQL_TEXT = @DROP_SQL_TEXT;
 
END
 
 
 
GO

# Create Stored Procedure USP_ANALYSE_SSIS_SQL_SP_TEXT

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/06
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_ANALYSE_SSIS_SQL_SP_TEXT]
	@PACKAGE_ANALYSIS_ID INT,
	@SP_SQL_TEXT NVARCHAR(MAX),
	@SOURCE_DB NVARCHAR(128)
AS
BEGIN
 
	SET NOCOUNT ON;
 
	 
	DECLARE 
		@DROP_SQL_TEXT NVARCHAR(MAX), 
		@INSERT_SQL_TEXT NVARCHAR(MAX),
		@SQL_EXEC_TEXT NVARCHAR(MAX), 
		@PARM_DEFINITION NVARCHAR(1000),
		@PARM_DEFINITION_PROC NVARCHAR(1000),
 
		@TEMP_SP_NAME NCHAR(36),
		@PACKAGE_NAME NVARCHAR(200),
		@DATA_FLOW_NAME NVARCHAR(200);
 
 
	SET @TEMP_SP_NAME = NEWID();    
	SET @SQL_EXEC_TEXT = CONCAT(N'EXEC ', @SOURCE_DB,'.sys.sp_executesql @SQL_TEXT');
	SET @PARM_DEFINITION = N'@SQL_TEXT NVARCHAR(MAX)';
	SET @PARM_DEFINITION_PROC = N'@TEMP_SP_NAME NVARCHAR(38)';
 
	SET @SP_SQL_TEXT = REPLACE(@SP_SQL_TEXT,N'?',N'''''');
 
	--Create temp SP
	SET @SP_SQL_TEXT = CONCAT(N'CREATE PROC test.', QUOTENAME(@TEMP_SP_NAME) , N' AS', CHAR(10) , CHAR(13) , @SP_SQL_TEXT);
	--PRINT @SP_SQL_TEXT
	EXEC sys.sp_executesql @SQL_EXEC_TEXT,@PARM_DEFINITION,@SQL_TEXT = @SP_SQL_TEXT;
  
	--DELETE FROM [dbo].[ETL_SSIS_PACKAGE_SQL_ANALYSIS]
	--WHERE [PACKAGE_ANALYSIS_ID] = @PACKAGE_ANALYSIS_ID;
	--Insert into sql task table
	SET @INSERT_SQL_TEXT = CONCAT(
	N'
	INSERT INTO [dbo].[ETL_SSIS_PACKAGE_SQL_TASK_ANALYSIS]
	SELECT DISTINCT ',@PACKAGE_ANALYSIS_ID,
	N'  
	,ISNULL(D.referenced_database_name,N''{SOURCE_DB}'') AS referenced_database_name,
	ISNULL(referenced_schema_name,N''dbo'') AS referenced_schema_name,
		referenced_entity_name 
	FROM 
	{SOURCE_DB}.sys.sql_expression_dependencies AS D
	INNER JOIN
	{SOURCE_DB}.sys.procedures AS V
	ON D.referencing_id = V.object_id
	WHERE V.name =''',@TEMP_SP_NAME,'''') ;
 
	SET @INSERT_SQL_TEXT = REPLACE(@INSERT_SQL_TEXT,N'{SOURCE_DB}',@SOURCE_DB);
 
	--PRINT @INSERT_SQL_TEXT;
 
	EXEC sys.sp_executesql @INSERT_SQL_TEXT,@PARM_DEFINITION_PROC,@TEMP_SP_NAME = @TEMP_SP_NAME;
	--Drop temp view
	SET @DROP_SQL_TEXT = N'DROP PROC test.' + QUOTENAME(@TEMP_SP_NAME);
	--PRINT(@DROP_SQL_TEXT);
	EXEC sys.sp_executesql @SQL_EXEC_TEXT,@PARM_DEFINITION,@SQL_TEXT = @DROP_SQL_TEXT;
 
 
END
 
 
 
GO


# Create Stored Procedure USP_EXEC_ANALYSE_SSIS_CONNECTION

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/06
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_EXEC_ANALYSE_SSIS_CONNECTION]
 
AS
BEGIN
 
	SET NOCOUNT ON;
 
	DECLARE @DOC_XML XML;
 
	TRUNCATE TABLE dbo.ETL_SSIS_CONNECTIONS;
 
	DECLARE EX_CUR CURSOR SCROLL
	FOR 
		SELECT TRY_CONVERT(XML, X.file_stream)
		FROM [ft].[ETL_SSIS_CONNECTIONS] AS X
		ORDER BY X.name;
 
	OPEN EX_CUR;
	FETCH NEXT FROM EX_CUR INTO @DOC_XML
	WHILE @@FETCH_STATUS = 0
	BEGIN
 
		 EXEC [dbo].[USP_ANALYSE_SSIS_CONNECTION] @DOC_XML = @DOC_XML;
 
		 FETCH NEXT FROM EX_CUR INTO @DOC_XML
 
	END
	CLOSE EX_CUR
	DEALLOCATE  EX_CUR;
 
 
 
END
GO

# Create Stored Procedure USP_EXEC_ANALYSE_SSIS_PACKAGE

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/06
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_EXEC_ANALYSE_SSIS_PACKAGE]
 
AS
BEGIN
 
	SET NOCOUNT ON;
 
	DECLARE @DOC_XML XML,@PACKAGE_NAME NVARCHAR(200);
 
	TRUNCATE TABLE dbo.ETL_SSIS_PACKAGE_ANALYSIS;
 
	DECLARE EX_CUR CURSOR SCROLL
	FOR 
		SELECT X.name,TRY_CONVERT(XML, X.file_stream)
		FROM [ft].[ETL_SSIS_PACKAGES] AS X
		ORDER BY X.name;
 
	OPEN EX_CUR;
	FETCH NEXT FROM EX_CUR INTO @PACKAGE_NAME,@DOC_XML
	WHILE @@FETCH_STATUS = 0
	BEGIN
 
		 EXEC [dbo].[USP_ANALYSE_SSIS_PACKAGE] @PACKAGE_NAME = @PACKAGE_NAME,@DOC_XML = @DOC_XML;
 
		 FETCH NEXT FROM EX_CUR INTO @PACKAGE_NAME,@DOC_XML
 
	END
	CLOSE EX_CUR
	DEALLOCATE  EX_CUR;
 
 
 
 
 
 
END
GO


# Create Stored Procedure USP_EXEC_ANALYSE_SSIS_SQL_SELECT_TEXT

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/06
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_EXEC_ANALYSE_SSIS_SQL_SELECT_TEXT]
AS
BEGIN
 
	SET NOCOUNT ON;
	DECLARE @PACKAGE_ANALYSIS_ID INT,@SOURCE_TEXT NVARCHAR(MAX),@SOURCE_DB NVARCHAR(128);
 
	TRUNCATE TABLE dbo.ETL_SSIS_PACKAGE_SQL_SELECT_ANALYSIS;
 
	DECLARE EX_CUR CURSOR SCROLL
	FOR
		--STG层的SOURCE SQL语句通常在外部服务器,暂不分析
		SELECT 
			PACKAGE_ANALYSIS_ID,
             SOURCE_TEXT,
			SOURCE_CONNECTION
		FROM dbo.ETL_SSIS_PACKAGE_ANALYSIS
		WHERE SOURCE_TYPE = N'SqlCommand'
		AND TARGET_CONNECTION IN (N'DW_EDW',N'DW_ADM')
		ORDER BY PACKAGE_ANALYSIS_ID;
		 
 
	OPEN EX_CUR;
	FETCH NEXT FROM EX_CUR INTO @PACKAGE_ANALYSIS_ID,@SOURCE_TEXT,@SOURCE_DB;
	WHILE @@FETCH_STATUS = 0
	BEGIN
 
		--执行单个语句分析
		EXEC [dbo].[USP_ANALYSE_SSIS_SQL_SELECT_TEXT] @PACKAGE_ANALYSIS_ID = @PACKAGE_ANALYSIS_ID, @SELECT_SQL_TEXT = @SOURCE_TEXT, @SOURCE_DB = @SOURCE_DB;
 
 
		FETCH NEXT FROM EX_CUR INTO @PACKAGE_ANALYSIS_ID,@SOURCE_TEXT,@SOURCE_DB;
 
	END
	CLOSE EX_CUR
	DEALLOCATE  EX_CUR;
 
 
 
 
 
 
END
GO


# Create Stored Procedure USP_EXEC_ANALYSE_SSIS_SQL_SP_TEXT

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/06
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_EXEC_ANALYSE_SSIS_SQL_SP_TEXT]
AS
BEGIN
 
	SET NOCOUNT ON;
	DECLARE @PACKAGE_ANALYSIS_ID INT,@SOURCE_TEXT NVARCHAR(MAX),@SOURCE_DB NVARCHAR(128);
 
	TRUNCATE TABLE dbo.ETL_SSIS_PACKAGE_SQL_TASK_ANALYSIS;
 
	DECLARE EX_CUR CURSOR SCROLL
	FOR
 
		SELECT 
			PACKAGE_ANALYSIS_ID,
             SOURCE_TEXT,
			SOURCE_CONNECTION
		FROM dbo.ETL_SSIS_PACKAGE_ANALYSIS
		WHERE SOURCE_TYPE = N'SQL Task'
		AND SOURCE_CONNECTION IN (N'DW_EDW',N'DW_ADM')
		AND SOURCE_TEXT LIKE N'%USP_INSERT%'
		ORDER BY PACKAGE_ANALYSIS_ID;
		 
 
	OPEN EX_CUR;
	FETCH NEXT FROM EX_CUR INTO @PACKAGE_ANALYSIS_ID,@SOURCE_TEXT,@SOURCE_DB;
	WHILE @@FETCH_STATUS = 0
	BEGIN
 
		--执行单个语句分析
		EXEC [dbo].[USP_ANALYSE_SSIS_SQL_SP_TEXT] @PACKAGE_ANALYSIS_ID = @PACKAGE_ANALYSIS_ID, @SP_SQL_TEXT = @SOURCE_TEXT, @SOURCE_DB = @SOURCE_DB;
 
 
		FETCH NEXT FROM EX_CUR INTO @PACKAGE_ANALYSIS_ID,@SOURCE_TEXT,@SOURCE_DB;
 
	END
	CLOSE EX_CUR
	DEALLOCATE  EX_CUR;
 
 
END
GO


# Create Stored Procedure USP_GEN_ALL_TABLE_VIEWS

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/07
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_GEN_ALL_TABLE_VIEWS]
	@STG_DB_NAME SYSNAME = 'DW_STG',
	@EDW_DB_NAME SYSNAME = 'DW_EDW',
	@DM_DB_NAME SYSNAME = 'DW_ADM',
	@DCI_DB_NAME SYSNAME = N'DW_DCI'
AS
BEGIN
 
	SET NOCOUNT ON;
 
	--获取DW 3层数据库+DCI所有Table/View
	--=============================================================================================
	DECLARE @STG_DB_ID INT, @EDW_DB_ID INT, @DM_DB_ID INT;
	DECLARE @SQL_TEXT NVARCHAR(MAX);
 
	SET @DM_DB_ID =
	(SELECT database_id FROM sys.databases WHERE name = @DM_DB_NAME);
 
	SET @EDW_DB_ID =
	(SELECT database_id FROM sys.databases WHERE name = @EDW_DB_NAME);
 
	SET @STG_DB_ID =
	(SELECT database_id FROM sys.databases WHERE name = @STG_DB_NAME);
 
	TRUNCATE TABLE [dbo].[ETL_ALL_TABLE_VIEWS];
 
	SET @SQL_TEXT = CONCAT(
	N'USE ',@DM_DB_NAME,
	N'
	INSERT INTO [DW_REP].[dbo].[ETL_ALL_TABLE_VIEWS]
	SELECT [TYPE_DESC],''',@DM_DB_NAME,N''',SCHEMA_NAME(schema_id) AS  SCHEMA_NAME, name AS TABLE_NAME
	FROM sys.objects
	WHERE TYPE IN (''U'',''V'') AND SCHEMA_NAME(schema_id) NOT IN (''sys'',''test'',''bak'',''his'',''ini'',''tmp'');');
 
	EXECUTE sp_executesql @SQL_TEXT;
	SET @SQL_TEXT = CONCAT(
	N'USE ',@EDW_DB_NAME,
	N'
	INSERT INTO [DW_REP].[dbo].[ETL_ALL_TABLE_VIEWS]
	SELECT [TYPE_DESC],''',@EDW_DB_NAME,N''',SCHEMA_NAME(schema_id) AS  SCHEMA_NAME, name AS TABLE_NAME
	FROM sys.objects
	WHERE TYPE IN (''U'',''V'') AND SCHEMA_NAME(schema_id) NOT IN (''sys'',''test'',''bak'',''his'',''ini'',''tmp'');');
 
	EXECUTE sp_executesql @SQL_TEXT;
 
	SET @SQL_TEXT = CONCAT(
	N'USE ',@STG_DB_NAME,
	N'
	INSERT INTO [DW_REP].[dbo].[ETL_ALL_TABLE_VIEWS]
	SELECT [TYPE_DESC],''',@STG_DB_NAME,N''',SCHEMA_NAME(schema_id) AS  SCHEMA_NAME, name AS TABLE_NAME
	FROM sys.objects
	WHERE TYPE IN (''U'',''V'') AND SCHEMA_NAME(schema_id) NOT IN (''sys'',''test'',''bak'',''his'',''ini'',''tmp'');');
 
	EXECUTE sp_executesql @SQL_TEXT;
 
	SET @SQL_TEXT = CONCAT(
	N'USE ',@DCI_DB_NAME,
	N'
	INSERT INTO [DW_REP].[dbo].[ETL_ALL_TABLE_VIEWS]
	SELECT [TYPE_DESC],''',@DCI_DB_NAME,N''',SCHEMA_NAME(schema_id) AS  SCHEMA_NAME, name AS TABLE_NAME
	FROM sys.objects
	WHERE TYPE IN (''U'',''V'') AND SCHEMA_NAME(schema_id) NOT IN (''sys'',''test'',''bak'',''his'',''ini'',''tmp'');');
 
	EXECUTE sp_executesql @SQL_TEXT;
 
END
GO


# Create Stored Procedure USP_GEN_SP_REFERENCED_TABLE_MAPPING

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/07
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_GEN_SP_REFERENCED_TABLE_MAPPING]
	@STG_DB_NAME SYSNAME = 'DW_STG',
	@EDW_DB_NAME SYSNAME = 'DW_EDW',
	@DM_DB_NAME SYSNAME = 'DW_ADM'
AS
BEGIN
 
	SET NOCOUNT ON;
 
	DECLARE @SQL_TEXT NVARCHAR(MAX);
 
     --检索存储过程与其他对象的依赖关系
	--=============================================================================================
	TRUNCATE TABLE dbo.[ETL_SP_REFERENCED_TABLE_MAPPING];
	SET @SQL_TEXT = CONCAT(
	N'USE ',@DM_DB_NAME,
	N'
	INSERT INTO [DW_REP].[dbo].[ETL_SP_REFERENCED_TABLE_MAPPING]
	SELECT 
		','''',@DM_DB_NAME,''',',
		'''dbo'',
		O.name AS SP_NAME,
	ISNULL(referenced_database_name,''',@DM_DB_NAME,'''),',
	N'    LOWER(ISNULL(referenced_schema_name,''dbo'')),  
		referenced_entity_name
	FROM 
	sys.sql_expression_dependencies AS sed  
	INNER JOIN 
	sys.procedures AS o 
	ON sed.referencing_id = o.object_id
 
	--仅检索dbo.USP_INSERT模式开头的SP
	WHERE SCHEMA_NAME(schema_id) = ''dbo''
	AND o.name LIKE N''USP_INSERT%'' ;');
 
	--PRINT(@SQL_TEXT);
	EXECUTE sp_executesql @SQL_TEXT;
 
	SET @SQL_TEXT = CONCAT(
	N'USE ',@EDW_DB_NAME,
	N'
	INSERT INTO [DW_REP].[dbo].[ETL_SP_REFERENCED_TABLE_MAPPING]
	SELECT 
		','''',@EDW_DB_NAME,''',',
		'''dbo'',
		O.name AS SP_NAME,
	ISNULL(referenced_database_name,''',@EDW_DB_NAME,'''),',
	N'    LOWER(ISNULL(referenced_schema_name,''dbo'')),  
		referenced_entity_name
	FROM 
	sys.sql_expression_dependencies AS sed  
	INNER JOIN 
	sys.procedures AS o 
	ON sed.referencing_id = o.object_id
 
	--仅检索dbo.USP_INSERT模式开头的SP
	WHERE SCHEMA_NAME(schema_id) = ''dbo''
	AND o.name LIKE N''USP_INSERT%'' ;');
 
	EXECUTE sp_executesql @SQL_TEXT;
 
 
END
GO


# Create Stored Procedure USP_GEN_SP_TARGET_TABLE_MAPPING

In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/07
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_GEN_SP_TARGET_TABLE_MAPPING]
	@STG_DB_NAME SYSNAME = 'DW_STG',
	@EDW_DB_NAME SYSNAME = 'DW_EDW',
	@DM_DB_NAME SYSNAME = 'DW_ADM'
AS
BEGIN
 
	SET NOCOUNT ON;
 
	DECLARE @SQL_TEXT NVARCHAR(MAX);
 
     TRUNCATE TABLE dbo.[ETL_SP_TARGET_TABLE_MAPPING];
 
	--一部分可以通过SP名称判断
	--以USP_INSERT开头模式检索
	--EDW
	SET @SQL_TEXT = CONCAT(
	N'USE ',@EDW_DB_NAME,
	N'
	INSERT INTO [DW_REP].[dbo].[ETL_SP_TARGET_TABLE_MAPPING]
	SELECT
		''',@EDW_DB_NAME,''',',
		'''dbo'',
		O.name,',
		'''',@EDW_DB_NAME,''',',
		N'N''dbo'',
		REPLACE(name,''USP_INSERT_'',''''),
		1 AS IS_SYSTEM,
		NULL AS COMMENT
	FROM sys.procedures AS O
	WHERE SCHEMA_NAME(schema_id) = ''dbo''
	AND o.name LIKE N''USP_INSERT%''' );
	--PRINT (@SQL_TEXT);
	EXECUTE sp_executesql @SQL_TEXT;
 
	--DM
	SET @SQL_TEXT = CONCAT(
	N'USE ',@DM_DB_NAME,
	N'
	INSERT INTO [DW_REP].[dbo].[ETL_SP_TARGET_TABLE_MAPPING]
	SELECT
		''',@DM_DB_NAME,''',',
		'''dbo'',
		O.name,',
		'''',@DM_DB_NAME,''',',
		N'N''dbo'',
		REPLACE(name,''USP_INSERT_'',''''),
		1 AS IS_SYSTEM,
		NULL AS COMMENT
	FROM sys.procedures AS O
	WHERE SCHEMA_NAME(schema_id) = ''dbo''
	AND o.name LIKE N''USP_INSERT%''' );
	--PRINT (@SQL_TEXT);
	EXECUTE sp_executesql @SQL_TEXT;
 
	--另一部分通过手工维护
	INSERT INTO dbo.[ETL_SP_TARGET_TABLE_MAPPING]
	SELECT
		T.SP_DATABASE_NAME,
		T.SP_SCHEMA_NAME,
		T.SP_NAME,
		T.TARGET_DATABASE_NAME,
		T.TARGET_SCHEMA_NAME,
		T.TARGET_TABLE_NAME,
		0 AS IS_SYSTEM,
		COMMENT AS COMMENT
	FROM dbo.[ETL_SP_MANUAL_TARGET_TABLE_MAPPING] AS T;
 
 
END
 
 
GO


# Create Stored Procedure USP_GEN_SSIS_PACKAGE_TABLE_LINEAGE
The SP to schedule execution pipeline

Steps|SP
--|:--:|
Generate all Tables/Views|[dbo].[USP_GEN_ALL_TABLE_VIEWS]
Generate relationship between SP and Target Tabl|[dbo].[USP_GEN_SP_TARGET_TABLE_MAPPING]
Analyzze tables/views in SPs|[dbo].[USP_GEN_SP_REFERENCED_TABLE_MAPPING]
Analyze SSIS Connection|[dbo].[USP_EXEC_ANALYSE_SSIS_CONNECTION]
Analyze SSIS Package|[dbo].[USP_EXEC_ANALYSE_SSIS_PACKAGE]
Analyze SSIS Package.DataFlow.Source SQL text|[dbo].[USP_EXEC_ANALYSE_SSIS_SQL_SELECT_TEXT]
Analyze SSIS Package.SQLTask.SQL text|[dbo].[USP_EXEC_ANALYSE_SSIS_SQL_SP_TEXT]


In [0]:
-- =============================================
-- Author:        Needn
-- Create date: 2019/11/07
-- Description:    
-- =============================================
CREATE PROCEDURE [dbo].[USP_GEN_SSIS_PACKAGE_TABLE_LINEAGE]
	@STG_DB_NAME SYSNAME = 'DW_STG',
	@EDW_DB_NAME SYSNAME = 'DW_EDW',
	@DM_DB_NAME SYSNAME = 'DW_ADM'
AS
BEGIN
 
	SET NOCOUNT ON;
 
	--生成所有Table/View
     EXEC [dbo].[USP_GEN_ALL_TABLE_VIEWS] @STG_DB_NAME = @STG_DB_NAME, @EDW_DB_NAME = @EDW_DB_NAME, @DM_DB_NAME = @DM_DB_NAME;
 
	--生成SP与Target Table的关系
	EXEC [dbo].[USP_GEN_SP_TARGET_TABLE_MAPPING] @STG_DB_NAME = @STG_DB_NAME, @EDW_DB_NAME = @EDW_DB_NAME, @DM_DB_NAME = @DM_DB_NAME;
 
	--生成存储过程与其他对象的依赖关系
	EXEC [dbo].[USP_GEN_SP_REFERENCED_TABLE_MAPPING] @STG_DB_NAME = @STG_DB_NAME, @EDW_DB_NAME = @EDW_DB_NAME, @DM_DB_NAME = @DM_DB_NAME;
 
	--解析SSIS Connection
	EXEC [dbo].[USP_EXEC_ANALYSE_SSIS_CONNECTION];
 
	--解析SSIS Package
	EXEC [dbo].[USP_EXEC_ANALYSE_SSIS_PACKAGE];
 
	--解析SSIS Package.DataFlow.Source的SQL文本
	EXEC [dbo].[USP_EXEC_ANALYSE_SSIS_SQL_SELECT_TEXT] ;
 
	--解析SSIS Package.SQLTask.SQL文本
	EXEC [dbo].[USP_EXEC_ANALYSE_SSIS_SQL_SP_TEXT];
 
	--结果
	--================================================================================================
	TRUNCATE TABLE [dbo].[ETL_SSIS_PACKAGE_TABLE_LINEAGE];
 
	--插入SP结果
	SELECT DISTINCT
		P.PACKAGE_NAME,
		P.CORE_COMPONENT_ID,
		P.CORE_COMPONENT_TYPE,
		SP.SP_DATABASE_NAME + '.' + SP.SP_SCHEMA_NAME + '.' + SP.SP_NAME AS CORE_COMPONENT_NAME, --P.CORE_COMPONENT_NAME,
		SP.REFERENCED_DATABASE_NAME,
		SP.REFERENCED_SCHEMA_NAME,
		SP.REFERENCED_TABLE_NAME,
		T.TARGET_DATABASE_NAME AS [REFERENCING_DATABASE_NAME],
         T.TARGET_SCHEMA_NAME AS [REFERENCING_SCHEMA_NAME],
		T.TARGET_TABLE_NAME AS [REFERENCING_TABLE_NAME],
		0 AS IS_MAIN_TABLE
	INTO #R_SP
	FROM 
	dbo.ETL_SSIS_PACKAGE_ANALYSIS AS P
	INNER JOIN
	dbo.[ETL_SSIS_PACKAGE_SQL_TASK_ANALYSIS] AS SPA
	ON P.PACKAGE_ANALYSIS_ID = SPA.PACKAGE_ANALYSIS_ID
	INNER JOIN
	dbo.ETL_SP_REFERENCED_TABLE_MAPPING AS SP
	ON SPA.REFERENCED_DATABASE_NAME = SP.SP_DATABASE_NAME AND
	   SPA.REFERENCED_SCHEMA_NAME = SP.SP_SCHEMA_NAME AND
	   SPA.REFERENCED_SP_NAME = SP.SP_NAME  
	INNER JOIN
	dbo.ETL_SP_TARGET_TABLE_MAPPING AS T
	ON SPA.REFERENCED_DATABASE_NAME = T.SP_DATABASE_NAME AND
	   SPA.REFERENCED_SCHEMA_NAME = T.SP_SCHEMA_NAME AND
	   SPA.REFERENCED_SP_NAME = T.SP_NAME      
	WHERE P.SOURCE_TYPE = N'SQL Task';
 
	--插入SSIS结果
	SELECT DISTINCT
		P.PACKAGE_NAME,
		P.CORE_COMPONENT_ID,
		P.CORE_COMPONENT_TYPE,
		P.CORE_COMPONENT_NAME,
	ISNULL(S.REFERENCED_DATABASE_NAME,ST.DATABASE_NAME) AS REFERENCED_DATABASE_NAME,
	ISNULL(S.REFERENCED_SCHEMA_NAME,ST.SCHEMA_NAME) AS REFERENCED_SCHEMA_NAME,
	ISNULL(S.REFERENCED_TABLE_NAME,ST.OBJECT_NAME) AS REFERENCED_TABLE_NAME,
		TT.DATABASE_NAME AS REFERENCING_DATABASE_NAME,
		TT.SCHEMA_NAME AS REFERENCING_SCHEMA_NAME,
		TT.OBJECT_NAME AS REFERENCING_TABLE_NAME,
		0 AS IS_MAIN_TABLE
	INTO #R_DF
	FROM 
	dbo.ETL_SSIS_PACKAGE_ANALYSIS AS P
	LEFT JOIN
	dbo.ETL_SSIS_PACKAGE_SQL_SELECT_ANALYSIS AS S
	ON P.PACKAGE_ANALYSIS_ID = S.PACKAGE_ANALYSIS_ID
	LEFT JOIN
	[dbo].[ETL_ALL_TABLE_VIEWS] AS ST
	ON P.SOURCE_TEXT = QUOTENAME(ST.SCHEMA_NAME) + '.' + QUOTENAME(ST.OBJECT_NAME) AND P.SOURCE_CONNECTION = ST.DATABASE_NAME
	LEFT JOIN
	[dbo].[ETL_ALL_TABLE_VIEWS] AS TT
	ON P.TARGET_TEXT = QUOTENAME(TT.SCHEMA_NAME) + '.' + QUOTENAME(TT.OBJECT_NAME) AND P.TARGET_CONNECTION = TT.DATABASE_NAME
 
	WHERE P.SOURCE_TYPE IN (N'SqlCommand',N'Table or view') AND
	TARGET_CONNECTION IN ('DW_EDW','DW_ADM');
	 
	WITH CTE AS
	(
		SELECT * FROM #R_SP UNION SELECT * FROM #R_DF
	)
	INSERT INTO dbo.[ETL_SSIS_PACKAGE_TABLE_LINEAGE]
	SELECT * FROM CTE AS T
	WHERE EXISTS
	--仅保留Table/View对象 
	(
		SELECT * FROM dbo.ETL_ALL_TABLE_VIEWS AS TV
		WHERE T.REFERENCED_DATABASE_NAME = TV.DATABASE_NAME AND T.REFERENCED_SCHEMA_NAME = TV.SCHEMA_NAME AND T.REFERENCED_TABLE_NAME = TV.OBJECT_NAME
	)
	AND EXISTS
	(
		SELECT * FROM dbo.ETL_ALL_TABLE_VIEWS AS TV2
		WHERE T.REFERENCING_DATABASE_NAME = TV2.DATABASE_NAME AND T.REFERENCING_SCHEMA_NAME = TV2.SCHEMA_NAME AND T.REFERENCING_TABLE_NAME = TV2.OBJECT_NAME
	)
	AND NOT (T.REFERENCED_DATABASE_NAME = T.REFERENCING_DATABASE_NAME AND T.REFERENCED_SCHEMA_NAME = T.REFERENCING_SCHEMA_NAME AND T.REFERENCED_TABLE_NAME = T.REFERENCING_TABLE_NAME);
END
 
GO
